In [1]:
# Import Packages
import numpy as np
import tensorflow as tf
import collections
import argparse
import time
import os
#from six.moves import cPickle
import pickle
print ("Packages Imported")

Packages Imported


In [2]:
# Load chars and vocab
load_dir = "data/linux_kernel"
with open(os.path.join(load_dir, 'chars_vocab.pkl'), 'rb') as f:
    chars, vocab = pickle.load(f, errors='ignore')
vocab_size = len(vocab) 
print ("'vocab_size' is %d" % (vocab_size))

'vocab_size' is 98


## Now, we are ready to make our RNN model with seq2seq
### This network is for sampling, so we don't need batches for sequenes nor optimizers

In [5]:
# Important RNN parameters 
rnn_size   = 128
num_layers = 2

batch_size = 1 # <= In the training phase, these were both 50
seq_length = 1

tf.reset_default_graph()

# Construct RNN model 
unitcell   = tf.contrib.rnn.BasicLSTMCell(rnn_size)
cell       = tf.contrib.rnn.MultiRNNCell([unitcell] * num_layers)
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])
istate     = cell.zero_state(batch_size, tf.float32)

# Weigths 
with tf.variable_scope('rnnlm'):
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    
    with tf.device("/cpu:0"):
        embedding = tf.get_variable("embedding", [vocab_size, rnn_size])
        inputs = tf.split( tf.nn.embedding_lookup(embedding, input_data), seq_length, 1)
        inputs = [tf.squeeze(_input, [1]) for _input in inputs]
        
outputs, final_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, istate, cell
                                          , loop_function=None, scope='rnnlm')
output = tf.reshape(tf.concat(outputs,1 ), [-1, rnn_size])

logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
probs  = tf.nn.softmax(logits)

print ("Network Ready")

Network Ready


In [6]:
# Restore RNN
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())
ckpt  = tf.train.get_checkpoint_state(load_dir)

print (ckpt.model_checkpoint_path)
saver.restore(sess, ckpt.model_checkpoint_path)


data/linux_kernel/model.ckpt-8000


# Finally, show what RNN has generated! 

In [8]:
# Sampling function
def weighted_pick(weights):
    t = np.cumsum(weights)
    s = np.sum(weights)
    return(int(np.searchsorted(t, np.random.rand(1)*s)))

# Sample using RNN and prime characters
prime = "/* "
state = sess.run(cell.zero_state(1, tf.float32))
for char in prime[:-1]:
    x = np.zeros((1, 1))
    x[0, 0] = vocab[char]
    state = sess.run(final_state, feed_dict={input_data: x, istate:state})

# Sample 'num' characters
ret  = prime
char = prime[-1] # <= This goes IN! 
num  = 1000
for n in range(num):
    x = np.zeros((1, 1))
    x[0, 0] = vocab[char]
    [probsval, state] = sess.run([probs, final_state]
        , feed_dict={input_data: x, istate:state})
    p      = probsval[0] 
    
    sample = weighted_pick(p)
    # sample = np.argmax(p)
    
    pred   = chars[sample]
    ret    = ret + pred
    char   = pred
    
print ("Sampling Done. \n___________________________________________\n")

print (ret)

Sampling Done. 
___________________________________________

/*      *Here the chain
 */
static void print_log_end(unsigned long interval,
			user[it.stats * sinfol_user, user);
cond_sig[dsubflate = this_leftimeoutcweinet].interrum = current->post;
EXPORT_SYMBOL_GPL(gf_prock, char *page)
{
	set_cpuset = {
		.spec = {				"earay_my_changed 0 -%%d", 5);
	/*
	 * If must mary for inode-added that arratible of (in reduse the subsystepes console, high table,
	 * the mask pos the signal expiue, do existing. This
	 * the systems searwards the needs memory attach to the markers.
	 */
	if (!timer.fm)
		break;

	switchreptimer *task;
	in (statt_time[bytes_and_cpu_timer_ticks);

	if (USTEX_TIMERS + subj->create->destion, 0);
	segindel_task = data;

	ret = sched_cntex = 0;
	struct file *mm_faylsge(pi_state, struct cfs_rq, new_name);
}

void sprefcount(struct module *mutex)
{
	int pidmad(p) >> task && klocks_set_modulest(f, 0)
		s->se.simbarct audit_tructure &&
     &module_module_entry(head));
}

vo

# Hope, it was good. 